In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
import csv
import random
import ast

from scipy import stats, integrate
import matplotlib.pyplot as plt
import re
import nltk
from nltk.probability import FreqDist

import seaborn as sns
sns.set(color_codes=True)

import scrapeMyProf as smp

%matplotlib inline

In [165]:
testRI

,pk_id,schoolcity_s,schoolcountry_s,schoolname_s,schoolstate_s,teacherfullname_s,url
0,202897.0,Kingston,United States,University of Rhode Island,RI,Susan Boatright-Horowitz,http://www.ratemyprofessors.com/ShowRatings.js...
1,102624.0,Kingston,United States,University of Rhode Island,RI,Art Mead,http://www.ratemyprofessors.com/ShowRatings.js...
2,1173437.0,Kingston,United States,University of Rhode Island,RI,Bradley Wetherbee,http://www.ratemyprofessors.com/ShowRatings.js...
3,571965.0,Kingston,United States,University of Rhode Island,RI,Albert Kausch,http://www.ratemyprofessors.com/ShowRatings.js...
4,385385.0,Kingston,United States,University of Rhode Island,RI,Michael McGregor,http://www.ratemyprofessors.com/ShowRatings.js...
5,69651.0,Kingston,United States,University of Rhode Island,RI,Anthony Mallilo,http://www.ratemyprofessors.com/ShowRatings.js...
6,243348.0,Kingston,United States,University of Rhode Island,RI,John Pantalone,http://www.ratemyprofessors.com/ShowRatings.js...
7,166478.0,Kingston,United States,University of Rhode Island,RI,Vladimir Dobrushkin,http://www.ratemyprofessors.com/ShowRatings.js...
8,886205.0,Kingston,United States,University of Rhode Island,RI,Susan Geldart,http://www.ratemyprofessors.com/ShowRatings.js...
9,20550.0,Kingston,United States,University of Rhode Island,RI,Cynthia Blanthorne,http://www.ratemyprofessors.com/ShowRatings.js...


In [9]:
ME_profs = smp.state_df('ME','../smp_data/ME_profs_021717')

File Output: ../smp_data/ME_profs_021717_proflist.csv


In [11]:
NH_profs = smp.state_df('NH','../smp_data/NH_profs_021917')

File Output: ../smp_data/NH_profs_021917_proflist.csv


In [7]:
ME_profs = pd.read_csv('../smp_data/ME_profs_022117_proflist.csv',encoding='iso-8859-1')

In [8]:
ME_reviews = smp.get_reviews(ME_profs,'../smp_data/ME_profs_022117')

Click
Click
Click
Click
Click
Complete Prof 218392 Time 06:46:38 02/25/17 EST
100 reviews
Added prof 218392 to ../smp_data/ME_profs_022117
Click
Click
Click
Click
Click
Complete Prof 166779 Time 06:46:48 02/25/17 EST
100 reviews
Added prof 166779 to ../smp_data/ME_profs_022117
Click
Click
Click
Click
Complete Prof 158650 Time 06:46:57 02/25/17 EST
96 reviews
Added prof 158650 to ../smp_data/ME_profs_022117
Click
Click
Click
Click
Click
Complete Prof 163049 Time 06:47:13 02/25/17 EST
114 reviews
Added prof 163049 to ../smp_data/ME_profs_022117
Click
Click
Click
Click
Complete Prof 171241 Time 06:47:23 02/25/17 EST
96 reviews
Added prof 171241 to ../smp_data/ME_profs_022117
Click
Click
Click
Click
Complete Prof 159769 Time 06:47:33 02/25/17 EST
96 reviews
Added prof 159769 to ../smp_data/ME_profs_022117
Click
Click
Click
Click
Click
Complete Prof 61373 Time 06:47:40 02/25/17 EST
100 reviews
Added prof 61373 to ../smp_data/ME_profs_022117
Click
Click
Click
Click
Click
Complete Prof 192012

In [ ]:
# This function takes an url with professor data and adds to a dataframe that holds to data for the school
# Starts with a df initially set up with columns
# this dataframe include professor IDs and URLS

def add_profs(url,df):
    school_result = urlopen(url)
    bsSchool = bs(school_result,"lxml").p.string
    
    #first delete returns and new lines
    noSpaceData = bsSchool.replace('\n','').replace('\r','').replace('í','i') +']'
    #this is to fix the fact that there is a name that contains double quotes around a nickname!
    noSpaceData = re.sub('\"[A-Za-z]+"(?= )',' ',noSpaceData)
    #get rid of multiple spaces (replace 2 or more with just 1)
    #noSpaceData = re.sub('  +','',noSpaceData)
    #get rid of the preamble junk
    noSpaceData = re.sub('^jQuery.*\[','[',noSpaceData)
    # Remove everything after the last ]
    noSpaceData = re.sub('\].*',']',noSpaceData)
    
    #Load it into a pandas dataframe 
    return df.append(ast.literal_eval(noSpaceData),ignore_index=True)